In [36]:
import collections
import sys
from functools import reduce
import json
from pprint import pprint
import os
import socket

import pandas as pd

from common import *
from vulnerability_database import VulnerabilityDatabase

#### Database generated with
```
LOAD=0.5 PORT=55555 node identification/aletheia-preprocessor.mjs
LD_LIBRARY_PATH=$DOLOSPY python -m dolospy preindexer --preprocessor-url http://localhost:55555/preprocess --worker $(nproc) --log-level INFO
```

#### Resultset generated with
```
# Last duration: 30 min
# Use --no-comp and adjusted output name for compartment-less analysis
LD_LIBRARY_PATH="$LD_LIBRARY_PATH:$VIRTUAL_ENV/lib/python3.12/site-packages/dolospy" PORT=4200 python aletheia_speed_eval_real_vd.py --worker $(nproc) -o $DATASETS/results-lab-aletheia.bson -s $DATASETS/object-storage.tar $DATASETS/bundles-daily/*
python aletheia_speed_eval_recover.py -o $DATASETS/real-bundles-results-aletheia.json -r $DATASETS/results-real-aletheia.bson $DATASETS/bundles-daily/*
```

In [37]:
with open(os.path.join(DATASETS, "real-bundles-results-aletheia.json"), "r") as f:
    data_with_compartments = json.load(f)
    
with open(os.path.join(DATASETS, "real-bundles-results-aletheia-nocomp.json"), "r") as f:
    data_without_compartments = json.load(f)

vulndb = VulnerabilityDatabase(os.path.join(DATASETS, "vulndb.json"))

In [38]:
del metric
def metric(similarity_dict):
    """
    Compute a similarity score

    :param similarity_dict: dict keys: "covered", "leftTotal", "rightTotal"
    :return: float
    """
    return similarity_dict["covered"] / similarity_dict["leftTotal"] if similarity_dict["leftTotal"] > 0 else 0

In [39]:
def generate_pnpm_list_for_bundlerstudy():
    pnpmPkgs = set()
    for result in data_with_compartments:
        if result.get("ignore", False):
            continue

        truths = set(reduce(extendReduce, [parse_pnpm_names(name) for name in result["groundTruth"]], []))
        if "noCompartments" in result: continue
        similarities = result["similarities"]
        for truth in truths:
            pkg, vers = truth.rsplit("@", 1)
            pnpmPkgs.add(pkg)

    with open(f"/tmp/pnpm.list", "w") as f:
        f.write("\n".join(sorted(pnpmPkgs)))

# Compartment results

In [75]:
def real_version_detection(resultset, /, nocomp=False):
    class Stats:
        pass

    stats = Stats()
    stats.total = 0

    # Metric 1
    stats.noError = 0
    stats.patchError = 0
    stats.minorError = 0
    stats.majorError = 0

    stats.hasNoError = collections.Counter()
    stats.hasNoErrorAndUnique = collections.Counter()
    stats.hasMajorError = collections.Counter()
    stats.hasMinorError = collections.Counter()
    stats.hasPatchError = collections.Counter()

    # Metric 2
    stats.versionDifferences = []

    # Metric 3
    stats.vulnerableTruePositive = 0
    stats.vulnerableTrueNegative = 0
    stats.vulnerableFalsePositive = 0
    stats.vulnerableFalseNegative = 0

    # Other
    stats.packages = set()

    for result in resultset:
        if result.get("ignore", False):
            continue

        truths = set(reduce(extendReduce, [parse_pnpm_names(name) for name in result["groundTruth"]], []))
        if not nocomp and "noCompartments" in result: continue
        similarities = result["similarities"]
        for truth in truths:
            pkg, vers = truth.rsplit("@", 1)

            # We ignore packages which are not indexed
            if pkg in similarities and len(similarities[pkg]) > 0:
                scores = {k: metric(v) for k, v in similarities[pkg].items()}

                try:
                    assert vers in scores, f"Impossible to detect version. Maybe DB or lab bundle dataset are not synced?\n{truth=} {scores=}"
                except AssertionError:
                    print(f"WARNING Skipping {truth}")
                    continue

                maxScore = max(scores.values())
                maxVersions = [k for k in scores.keys() if scores[k] == maxScore]

                stats.total += 1
                stats.packages.add(pkg)

                try:
                    distance = semver_distance_list(vers, list(maxVersions))
                    stats.versionDifferences.append(distance)
                except ValueError as e:
                    print(result.get("domain"), e)
                    continue

                if vers in maxVersions:
                    stats.noError += 1
                    stats.hasNoError.update({pkg: 1})
                    if len(maxVersions) == 1:
                        stats.hasNoErrorAndUnique.update({pkg: 1})

                else:
                    if pkg == "vfile":
                        print(f"INFO vfile: {result['domain']}")

                    if distance[0] > 0:
                        stats.majorError += 1
                        stats.minorError += 1
                        stats.patchError += 1
                        stats.hasMajorError.update({pkg: 1})
                        stats.hasMinorError.update({pkg: 1})
                        stats.hasPatchError.update({pkg: 1})
                    elif distance[1] > 0:
                        stats.minorError += 1
                        stats.patchError += 1
                        stats.hasMinorError.update({pkg: 1})
                        stats.hasPatchError.update({pkg: 1})
                    elif distance[2] > 0:
                        stats.patchError += 1
                        stats.hasPatchError.update({pkg: 1})

                    detected_vulns = [vulndb.is_vulnerable(pkg, v) for v in maxVersions]
                    try:
                        if vulndb.is_vulnerable(pkg, vers):
                            if all(detected_vulns):
                                stats.vulnerableTruePositive += 1
                            elif all([not v for v in detected_vulns]):
                                stats.vulnerableFalseNegative += 1
                        else:
                            if all(detected_vulns):
                                stats.vulnerableFalsePositive += 1
                            elif all([not v for v in detected_vulns]):
                                stats.vulnerableTrueNegative += 1
                    except ValueError:
                        pass
    return stats

In [ ]:
stats = stats_comp = real_version_detection(data_with_compartments)

In [42]:
print(f"Total results: {stats.total}")
print("")
print("Metric 1:")
print(f"  No Error: {stats.noError}")
print(f"  Major Error: {stats.majorError}")
print(f"  Minor Error: {stats.minorError}")
print(f"  Patch Error: {stats.patchError}")
print("")
print("Metric 2:")
print(f"  Major Error (min/median/mean/max): {'/'.join(map(str, compute_statistics([d[0] for d in stats.versionDifferences])))}")
print(f"  Minor Error (min/median/mean/max): {'/'.join(map(str, compute_statistics([d[1] for d in stats.versionDifferences])))}")
print(f"  Patch Error (min/median/mean/max): {'/'.join(map(str, compute_statistics([d[2] for d in stats.versionDifferences])))}")
print("")
print("Metric 3:")
print(f"  TP / FN: {stats.vulnerableTruePositive:4}  {stats.vulnerableFalseNegative:4}")
print(f"  FP / TN: {stats.vulnerableFalsePositive:4}  {stats.vulnerableTrueNegative:4}")

Total results: 7128

Metric 1:
  No Error: 5855
  Major Error: 433
  Minor Error: 947
  Patch Error: 1273

Metric 2:
  Major Error (min/median/mean/max): 0/0.0/0.1026936026936027/8
  Minor Error (min/median/mean/max): 0/0.0/1.8063973063973064/273
  Patch Error (min/median/mean/max): 0/0.0/0.44556677890011226/25

Metric 3:
  TP / FN:   38     7
  FP / TN:   91  1132


In [43]:
print("Distribution of package errors:")
major = [1 - stats.hasMajorError.get(pkg, 0) / (stats.hasNoError.get(pkg, 0) + stats.hasPatchError.get(pkg, 0)) for pkg in stats.packages]
minor = [1 - stats.hasMinorError.get(pkg, 0) / (stats.hasNoError.get(pkg, 0) + stats.hasPatchError.get(pkg, 0)) for pkg in stats.packages]
patch = [1 - stats.hasPatchError.get(pkg, 0) / (stats.hasNoError.get(pkg, 0) + stats.hasPatchError.get(pkg, 0)) for pkg in stats.packages]
print(f"Count perfect major: {sum(1 for m in major if m >= 0.99)}/{len(major)}")
print(f"Count perfect minor: {sum(1 for m in minor if m >= 0.99)}/{len(minor)}")
print(f"Count perfect patch: {sum(1 for m in patch if m >= 0.99)}/{len(patch)}")

Distribution of package errors:
Count perfect major: 672/726
Count perfect minor: 601/726
Count perfect patch: 559/726


# Results w/o Compartments

In [ ]:
stats = stats_nocomp = real_version_detection(data_without_compartments, nocomp=True)

In [46]:
print(f"Total results: {stats.total}")
print("")
print("Metric 1:")
print(f"  No Error: {stats.noError}")
print(f"  Major Error: {stats.majorError}")
print(f"  Minor Error: {stats.minorError}")
print(f"  Patch Error: {stats.patchError}")
print("")
print("Metric 2:")
print(f"  Major Error (min/median/mean/max): {'/'.join(map(str, compute_statistics([d[0] for d in stats.versionDifferences])))}")
print(f"  Minor Error (min/median/mean/max): {'/'.join(map(str, compute_statistics([d[1] for d in stats.versionDifferences])))}")
print(f"  Patch Error (min/median/mean/max): {'/'.join(map(str, compute_statistics([d[2] for d in stats.versionDifferences])))}")
print("")
print("Metric 3:")
print(f"  TP / FN: {stats.vulnerableTruePositive:4}  {stats.vulnerableFalseNegative:4}")
print(f"  FP / TN: {stats.vulnerableFalsePositive:4}  {stats.vulnerableTrueNegative:4}")
print("Distribution of package errors:")
major = [1 - stats.hasMajorError.get(pkg, 0) / (stats.hasNoError.get(pkg, 0) + stats.hasPatchError.get(pkg, 0)) for pkg in stats.packages]
minor = [1 - stats.hasMinorError.get(pkg, 0) / (stats.hasNoError.get(pkg, 0) + stats.hasPatchError.get(pkg, 0)) for pkg in stats.packages]
patch = [1 - stats.hasPatchError.get(pkg, 0) / (stats.hasNoError.get(pkg, 0) + stats.hasPatchError.get(pkg, 0)) for pkg in stats.packages]
print(f"Count perfect major: {sum(1 for m in major if m >= 0.99)}/{len(major)}")
print(f"Count perfect minor: {sum(1 for m in minor if m >= 0.99)}/{len(minor)}")
print(f"Count perfect patch: {sum(1 for m in patch if m >= 0.99)}/{len(patch)}")

Total results: 17162

Metric 1:
  No Error: 11916
  Major Error: 2277
  Minor Error: 4343
  Patch Error: 5246

Metric 2:
  Major Error (min/median/mean/max): 0/0.0/0.4221535951520802/18
  Minor Error (min/median/mean/max): 0/0.0/2.50839063046265/894
  Patch Error (min/median/mean/max): 0/0.0/0.88993124344482/89

Metric 3:
  TP / FN:  114    89
  FP / TN:  304  4578
Distribution of package errors:
Count perfect major: 1027/1234
Count perfect minor: 846/1234
Count perfect patch: 764/1234
